In [1]:
'''given a url, scrape the article information'''

from newspaper import Article
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import openai

def process_complicated_author(author):

    with open('../openai_auth.json', 'r') as json_file:
        json_load = json.load(json_file)

    openai.api_key = json_load["api_key"]

    prompt = f"""
    Please give me the names of the authors included in this text, and remove any unnecessary spaces within names. Please give me the names, seperated by a space, and nothing else. For example, if the text was "求是》杂志记者 何雯雯 新县融媒体中心记者 韩 燕", I would want you to tell me "何雯雯 韩燕". The text is: {author}
    """

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[{
            "role":"user",
            "content":prompt
        }]
    )

    return response.choices[0].message.content

class QiuShiArticleParser:
    def __init__(self, url):
        # get the beautiful soup version
        self.url = url
        try:
            page = urlopen(url)
            html = page.read().decode("utf-8")
            self.soup = BeautifulSoup(html, "html.parser")

            # get the newspaper article version
            self.article = Article(url, language='zh')
            self.article.download()
            self.article.parse()
        except:
            print(f"Article with URL: {url} failed to parse")
            self.soup = None
            self.article = None

    def get_authors(self):
        if self.soup == None:
            return None
        span_els = self.soup.find_all("span", class_="appellation")
        try:
            # author is the second appel element
            author_el = span_els[1].text.strip()
            # remove the "Author:" piece
            author = author_el.split("：")[1]

            # see if it needs additional processing
            # no spaces, should be good to go. Or if None, return
            if author == None:
                return [author]
            elif " " not in author:
                return [author]
            # spaces, but length three - a 2 char name with a space
            elif len(author) == 3:
                return [author[0] + author[2]]
            # something more complicated is going on - could be multiple authors, could have additional info 
            else:
                print("Calling ChatGPT...")
                try:
                    author = process_complicated_author(author)
                    print(f"Author returned by ChatGPT: {author}")
                    # split the result in case there is more than one author
                    return author.split(" ")
                except:
                    print("ChatGPT call failed")
                return "CHATGPT" + author

        except:
            return None
    
    def get_title(self):
        try:
            return self.soup.find('h1').text.strip()
        except:
            return None
    
    def get_year_edition(self):
        try: 
            span_els = self.soup.find_all("span", class_="appellation")
            # year/edition is the first appel element
            ye_text = span_els[0].text.strip()
            # get the last piece with the date
            cleaned_ye_text = ye_text[-7:]

            year = cleaned_ye_text[:4]
            edition = cleaned_ye_text[-2:]

            return year, edition
        except:
            return None, None
    
    def get_date(self):
        try: 
            date_el = self.soup.find("span", class_="pubtime").text.strip()
            # remove the time piece
            date = date_el.split(" ")[0]
            return date
        except:
            return None
    
    def get_url(self):
        return self.url
    
    def get_text(self):
        try: 
            return self.article.text.replace('\n', "")
        except:
            return None

    
class QiuShiArticle:
    def __init__(self, article_parser):
        self.authors = article_parser.get_authors()
        self.title = article_parser.get_title()
        self.date = article_parser.get_date()
        self.year, self.edition = article_parser.get_year_edition()
        self.url = article_parser.get_url()

        try:
            self.text = article_parser.get_text()
        except:
            self.text = None
        

    def toJson(self):
        return self.__dict__

In [4]:
import pandas as pd
links_df = pd.read_csv('./links.csv')
links_df.head()

/var/folders/r4/qqn8tv4d7g18t_5fsxt6y1340000gn/T/ipykernel_2589/2423500418.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,year_name,edition_name,link
0,《求是》2024年,2024年第1期《求是》目录,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...
1,《求是》2024年,2024年第1期《求是》目录,http://www.qstheory.cn/dukan/qs/2023-12/31/c_1...
2,《求是》2024年,2024年第1期《求是》目录,http://www.qstheory.cn/dukan/qs/2023-12/31/c_1...
3,《求是》2024年,2024年第1期《求是》目录,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...
4,《求是》2024年,2024年第1期《求是》目录,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...


In [5]:
links_df = links_df.drop(columns=["year_name", "edition_name"])
print(len(links_df))
links_df.head()

2190


,link
0,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...
1,http://www.qstheory.cn/dukan/qs/2023-12/31/c_1...
2,http://www.qstheory.cn/dukan/qs/2023-12/31/c_1...
3,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...
4,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...


In [6]:
sample_articles_df = pd.DataFrame(columns=["authors", "title", "date", "year", "edition", "url", "text"])

In [7]:
from tqdm import tqdm

In [8]:
for link in tqdm(links_df["link"]):
    article_parser = QiuShiArticleParser(link)
    article = QiuShiArticle(article_parser)

    sample_articles_df.loc[len(sample_articles_df)] = [article.authors, article.title, article.date, article.year, article.edition, article.url, article.text]


  0%|          | 0/2190 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/r4/qqn8tv4d7g18t_5fsxt6y1340000gn/T/jieba.cache
Loading model cost 0.311 seconds.
Prefix dict has been built successfully.
  2%|▏         | 45/2190 [00:58<39:24,  1.10s/it]  

Calling ChatGPT...


  2%|▏         | 46/2190 [01:01<57:18,  1.60s/it]

Author returned by ChatGPT: 周昭成 陈聪
Calling ChatGPT...


...

Author returned by ChatGPT: 宗言
Calling ChatGPT...


 51%|█████     | 1109/2190 [51:57<25:29,  1.42s/it]

Author returned by ChatGPT: 高天鼎


 51%|█████     | 1113/2190 [52:06<50:25,  2.81s/it]


AttributeError: 'NoneType' object has no attribute 'text'

In [9]:
len(sample_articles_df)

1113

In [10]:
sample_articles_df.head()

,authors,title,date,year,edition,url,text
0,None,本期导读,2024-01-01,2024,01,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,本期发表了习近平总书记的重要文章《以美丽中国建设全面推进人与自然和谐共生的现代化》。文章强调...
1,[习近平],以美丽中国建设全面推进人与自然和谐共生的现代化,2023-12-31,2024,01,http://www.qstheory.cn/dukan/qs/2023-12/31/c_1...,以美丽中国建设全面推进人与自然和谐共生的现代化※习近平今后5年是美丽中国建设的重要时期，要深...
2,[《求是》杂志编辑部],全面推进美丽中国建设的系统部署,2023-12-31,2024,01,http://www.qstheory.cn/dukan/qs/2023-12/31/c_1...,全面推进美丽中国建设的系统部署《求是》杂志编辑部“今后5年是美丽中国建设的重要时期，要深入贯...
3,[中共政协全国委员会机关党组],为美丽中国建设贡献政协智慧和力量,2024-01-01,2024,01,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,为美丽中国建设贡献政协智慧和力量中共政协全国委员会机关党组党的二十大对推动绿色发展、促进人与...
4,[自然资源部],抓好海洋资源开发保护 为建设美丽中国提供蓝色动力,2024-01-01,2024,01,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,抓好海洋资源开发保护 为建设美丽中国提供蓝色动力自然资源部海洋是支撑未来发展的资源宝库和战略...


In [11]:
sample_articles_df.to_csv("articles.csv", index=False)

## Try picking up from where it left off

In [24]:
articles_df = pd.read_csv("articles.csv")

In [25]:
len(articles_df)

1113

In [21]:
links_df = links_df.iloc[1114:, :]

In [26]:
len(links_df)

1076

In [27]:
for link in tqdm(links_df["link"]):
    article_parser = QiuShiArticleParser(link)
    article = QiuShiArticle(article_parser)

    articles_df.loc[len(articles_df)] = [article.authors, article.title, article.date, article.year, article.edition, article.url, article.text]


  1%|          | 6/1076 [00:07<20:49,  1.17s/it]

Calling ChatGPT...


  1%|          | 7/1076 [00:09<23:48,  1.34s/it]

Author returned by ChatGPT: 国务院港澳事务办公室


  8%|▊         | 85/1076 [01:49<18:36,  1.13s/it]

...

 99%|█████████▉| 1067/1076 [23:06<00:09,  1.07s/it]

Calling ChatGPT...


 99%|█████████▉| 1068/1076 [23:07<00:09,  1.22s/it]

Author returned by ChatGPT: 袁誉柏 王建武


100%|██████████| 1076/1076 [23:17<00:00,  1.30s/it]


In [28]:
len(articles_df)

2189

In [29]:
articles_df.to_csv("all_articles.csv", index=False)

## Fix two missing articles

In [29]:
missing_articles = ['http://www.qstheory.cn/dukan/qs/2023-01/31/c_1129324011.htm', 'http://www.qstheory.cn/dukan/qs/2023-06/16/c_1129695716.htm']

In [30]:
import pandas as pd

# import the previously run results
articles = pd.read_csv('all_articles.csv')
len(articles)

2191

In [21]:
# drop the two articles that did not got loaded correctly before
articles = articles[(articles['url'] != missing_articles[0]) & (articles['url'] != missing_articles[1])]
len(articles)

2187

In [ ]:
# write it back
articles.to_csv('all_articles.csv', index=False)

In [25]:
# re-run them and add them to a new df

missing_articles_df = pd.DataFrame(columns=["authors", "title", "date", "year", "edition", "url", "text"])

for link in missing_articles:
    article_parser = QiuShiArticleParser(link)
    article = QiuShiArticle(article_parser)
    print(article.__dict__)

    missing_articles_df.loc[len(missing_articles_df)] = [article.authors, article.title, article.date, article.year, article.edition, article.url, article.text]


{'authors': ['《求是》杂志编辑部'], 'title': '把党的伟大自我革命进行到底', 'date': '2023-01-31', 'year': '2023', 'edition': '03', 'url': 'http://www.qstheory.cn/dukan/qs/2023-01/31/c_1129324011.htm', 'text': '把党的伟大自我革命进行到底《求是》杂志编辑部“要站在事关党长期执政、国家长治久安、人民幸福安康的高度，把全面从严治党作为党的长期战略、永恒课题，始终坚持问题导向，保持战略定力，发扬彻底的自我革命精神，永远吹冲锋号，把严的基调、严的措施、严的氛围长期坚持下去，把党的伟大自我革命进行到底。”在二十届中央纪委二次全会上，习近平总书记从新时代新征程党和国家事业发展全局的高度，深刻阐明健全全面从严治党体系的目标任务、实践要求，对坚定不移深入推进全面从严治党作出战略部署。治国必先治党，党兴才能国强。党的十八大以来，习近平总书记发表一系列重要讲话、提出一系列重要思想和重要要求，为深入推进全面从严治党指明前进方向、提供根本遵循。以习近平同志为核心的党中央把全面从严治党纳入“四个全面”战略布局，刀刃向内、刮骨疗毒，猛药祛疴、重典治乱，管党治党宽松软状况得到根本扭转，党在革命性锻造中更加坚强有力，开辟了百年大党自我革命的新境界，引领保障党和国家事业取得历史性成就、发生历史性变革。《全面从严治党探索出依靠党的自我革命跳出历史周期率的成功路径》一文，是习近平总书记2022年1月18日在十九届中央纪委六次全会上重要讲话的一部分。在这篇重要文章中，习近平总书记深刻阐述全面从严治党取得的历史性开创性成就、产生的全方位深层次影响，深刻总结新时代党的自我革命的成功实践，精辟概括对建设什么样的长期执政的马克思主义政党、怎样建设长期执政的马克思主义政党的规律性认识，为推进新时代党的建设新的伟大工程提供了基本遵循。要深刻学习领会这篇重要文章，学习好贯彻好习近平总书记党的二十大以来的一系列重要讲话精神，学习好贯彻好二十届中央纪委二次全会等一系列重要会议精神，切实把党的二十大精神学深悟透、融会贯通、落实落地，以党的伟大自我革命引领伟大社会革命，为全面建设社会主义现代化国家开好局起好步提供坚强保障。跳出历史周期率的第二个答案“我经常讲跳出历史周

In [26]:
missing_articles_df

,authors,title,date,year,edition,url,text
0,[《求是》杂志编辑部],把党的伟大自我革命进行到底,2023-01-31,2023,03,http://www.qstheory.cn/dukan/qs/2023-01/31/c_1...,把党的伟大自我革命进行到底《求是》杂志编辑部“要站在事关党长期执政、国家长治久安、人民幸福安...
1,None,时代楷模：敦煌研究院文物保护利用群体,2023-06-16,2023,12,http://www.qstheory.cn/dukan/qs/2023-06/16/c_1...,时代楷模：敦煌研究院文物保护利用群体图为敦煌研究院文物保护利用群体代表在“时代楷模”发布仪式...


In [27]:
missing_articles_df.to_csv('missing_articles.csv', index=False)

In [28]:
# merge the two CSV files
with open('missing_articles.csv', 'r') as f1:
    missing_articles = f1.read()

with open('all_articles.csv', 'a') as f2:
    f2.write('\n')
    f2.write(missing_articles)


after this the file had a few issues, and I ended up going through and deleting the link-only entries that I had confirmed were invalid, and finalizing the addition of the new articles to the end. so it was not totally programatic, which is obviously less than ideal.